# Project 8 : Déployer un modèle dans le cloud
## Partie 1 : Préparation des données localement
Le jeu de données est accessible ici : https://www.kaggle.com/moltean/fruits
Le but de ce notebook est juste de prendre un échantillon des images et de les uploader dans Amazon S3, elles seront dans la suite traitée par une architecture cloud

In [1]:
import os
import random
import boto3

Dans une première étape, on va juste corriger les dossier contenant les images, dans le but de les utiliser par la suite, pour ne pas avoir des erreurs à cause des espaces en trop

In [2]:
root = 'fruits-360/Training'

for dir_name in os.listdir(root):
    if ' ' in dir_name:
        new_dir_name = dir_name.replace(' ', '_')
        os.rename(os.path.join(root, dir_name), os.path.join(root, new_dir_name))
        print('Renaming {} to {}'.format(dir_name, new_dir_name))

Renaming Apple Braeburn to Apple_Braeburn
Renaming Apple Crimson Snow to Apple_Crimson_Snow
Renaming Apple Golden 1 to Apple_Golden_1
Renaming Apple Golden 2 to Apple_Golden_2
Renaming Apple Golden 3 to Apple_Golden_3
Renaming Apple Granny Smith to Apple_Granny_Smith
Renaming Apple Pink Lady to Apple_Pink_Lady
Renaming Apple Red 1 to Apple_Red_1
Renaming Apple Red 2 to Apple_Red_2
Renaming Apple Red 3 to Apple_Red_3
Renaming Apple Red Delicious to Apple_Red_Delicious
Renaming Apple Red Yellow 1 to Apple_Red_Yellow_1
Renaming Apple Red Yellow 2 to Apple_Red_Yellow_2
Renaming Avocado ripe to Avocado_ripe
Renaming Banana Lady Finger to Banana_Lady_Finger
Renaming Banana Red to Banana_Red
Renaming Cactus fruit to Cactus_fruit
Renaming Cantaloupe 1 to Cantaloupe_1
Renaming Cantaloupe 2 to Cantaloupe_2
Renaming Cherry 1 to Cherry_1
Renaming Cherry 2 to Cherry_2
Renaming Cherry Rainier to Cherry_Rainier
Renaming Cherry Wax Black to Cherry_Wax_Black
Renaming Cherry Wax Red to Cherry_Wax_Red
Re

Dans une seconde étape, on crée notre bucket S3 (on a utilisé la région us-east-2 parce que c'était la plus intéressante niveau prix, et parce qu'on ne va pas travailler sur un gros set de données, dans l'idéal ça dépendera des utilisateurs / des services avec lesquelles va communiquer l'application, et être au plus proches de ceux là)

In [3]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'oc-p8-salah'
region = 'us-east-2'
if not s3_resource.Bucket(bucket_name) in s3_resource.buckets.all():
    print('Creating bucket {}'.format(bucket_name))
    s3_resource.create_bucket(Bucket=bucket_name,
                              CreateBucketConfiguration={
                                  'LocationConstraint': region})

Creating bucket oc-p8-salah


On va choisir aléatoirement 10 produits, et pour chaque produit on va choisir aléatoirement 10 images, ce qui fait 100 images en tout, le but étant de faire juste une démonstration, mais faire plus et bien entendu possible, mais peux engendrer des coûts supplémentaires

In [4]:
sample_fruits = os.listdir(root)
random.shuffle(sample_fruits)
max_dirs_to_upload = 10
max_samples_per_dir = 10

uploaded_dirs = 1
for dir_name in sample_fruits:
    if uploaded_dirs >= max_samples_per_dir:
        print('Reaching max dirs to upload for the demo {} = '.format(max_dirs_to_upload))
        break
    uploaded_dirs = uploaded_dirs + 1
    print('Uploading sample of directory {}'.format(dir_name))
    label = dir_name
    if dir_name[-1].isnumeric():
        label = dir_name[0:-2]
        print('Using Label {} instead of {}'.format(label, dir_name))
    uploaded_samples = 0
    sample_unit = os.listdir(os.path.join(root, dir_name))
    random.shuffle(sample_unit)
    for img in sample_unit:
        if uploaded_samples >= max_samples_per_dir:
            print('Reaching max images to upload for this directory for the demo {} = '.format(max_samples_per_dir))
            break
        uploaded_samples = uploaded_samples + 1
        s3_client.upload_file(os.path.join(root, dir_name, img), bucket_name, label + '/' + img)
        print('Uploading img {} to s3'.format(os.path.join(root, dir_name, img)))

Uploading sample of directory Peach_Flat
Uploading img fruits-360/Training\Peach_Flat\181_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_315_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_309_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_226_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_185_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_182_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_248_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\190_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\r_308_100.jpg to s3
Uploading img fruits-360/Training\Peach_Flat\31_100.jpg to s3
Reaching max images to upload for this directory for the demo 10 = 
Uploading sample of directory Plum_3
Using Label Plum instead of Plum_3
Uploading img fruits-360/Training\Plum_3\r2_184_100.jpg to s3
Uploading img fruits-360/Training\Plum_3\r_65_100.jpg to s3
Uploading img fruits-360/Training\Plum_3\r_253_100.jpg

Et voilà, nos données sont disponibles sur le S3, reste plus qu'à en extraire les features, ce qui se fera dans le prochain notebook et qui a bien entendu déployé sur le cloud. 